In [1]:
# IMPORT 

import csv
from biomart import BiomartServer
import pandas as pd
import openpyxl

In [2]:
# DATASET

## biomart_server
server = BiomartServer('http://www.ensembl.org/biomart')
dataset = server.datasets['mmusculus_gene_ensembl']

In [3]:
# FUNCTIONS

def gene_dictionary(index,
                    gene_name,
                    gene_list_number,
                    gene_list_id,
                    source,
                    ensembl_gene_id,
                    ensembl_transcript_id,
                    refseq_mrna,
                    hgnc_symbol,
                    organism,
                    alias,
                    direction_string):
    
    dictionary = {
        'Index' : index,
        'Gene_name' : gene_name,
        'Gene_list_number' : gene_list_number,
        'Gene_list_index' : gene_list_id,
        'Source' : source,
        'Ensembl_gene_id' : ensembl_gene_id,
        'Ensembl_transcript_id' : ensembl_transcript_id,
        'RefSeq_mRNA_id' : refseq_mrna,
        'Organism' : organism,
        'HGNC_symbol' : hgnc_symbol,
        'Alias' : alias,
        'Info' : direction_string
    }
    
    return dictionary


  
def biomartParameters(mgi_symbol):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']

    filters = {'mgi_symbol':[mgi_symbol]}               # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values  

def biomartParametersbyEnsembl(ensembl_id):

    attributes = ['external_gene_name',
                  'ensembl_transcript_id',
                  'refseq_mrna']

    filters = {'ensembl_gene_id':[ensembl_gene_id]}               # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values  


def biomartHumanOrthologs(mgi_symbol):
    attributes = ['hsapiens_homolog_associated_gene_name']
    filters = {'mgi_symbol':[mgi_symbol]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values 

def biomartHumanOrthologsbyEnsembl(ensembl_id):

    attributes = ['hsapiens_homolog_associated_gene_name']

    filters = {'ensembl_gene_id':[ensembl_gene_id]}               # gene_name = mgi_symbol
    response = dataset.search({'attributes':attributes,'filters':filters})
    

    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values  


def alias_and_official(ls_notResponse,ls_row_10,ls_row_2):

    ls_response = []
    ls_response2 = []
    
    for notResponse in ls_notResponse:
        word = str(notResponse).lower()

        for i in range(len(ls_row_10)):
            if ls_row_10[i]:
                temp = str(ls_row_10[i]).lower()
                t_strings = temp.split("|")
                if word in t_strings:
                    ls_response.append([word, i, ls_row_1[i]])

    ls_response.insert(0, ['gene_name','alias_index','mgi_id'])         # print(len(ls_response))


    for notResponse in ls_notResponse:
        word = str(notResponse).lower()

        for i in range(len(ls_row_10)):
            if ls_row_10[i]:
                temp = str(ls_row_10[i]).lower()
                if word == temp:
                    ls_response2.append([word, i, ls_row_1[i]])
                    

    ls_response.insert(0, ['gene_name','official_index','mgi_id'])      # print(len(ls_response2))

    ls_response_3 = ls_response
    
    for response in ls_response2:
        if response not in ls_response_3:
            ls_response_3.append(response)
        else: print(response)

    ls_response_3.pop(0)
    ls_response_3.pop(0)
    ls_response.insert(0, ['gene_name','official/alias_index','mgi_id'])
    
    return ls_response_3

def biomartParameters_mgi(mgi_id):

    attributes = ['ensembl_gene_id',
                  'ensembl_transcript_id',
                  'refseq_mrna']
    filters = {'mgi_id':[mgi_id]}                     # gene_name = mgi_id
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values


def biomartHumanOrthologs_mgi(mgi_id):
    attributes = ['hsapiens_homolog_associated_gene_name']
    
    filters = {'mgi_id':[mgi_id]}
    response = dataset.search({'attributes':attributes,'filters':filters})
    
    # response_convertion
    values = [line.split("\t") for line in response.text.split("\n") if line.strip()]
        
    return values 


def updateCellswithAlias(mgi_file_path, dictionary_file_path, file_path):
    
    # Load MGI file and open workbook
    wb_mgi = openpyxl.load_workbook(mgi_file_path)
    ws_mgi = wb_mgi.active

    ls_row_alias = [ws_mgi.cell(row=i,column=10).value for i in range(2,ws_mgi.max_row+1)]
    ls_row_g_name = [ws_mgi.cell(row=i,column=2).value for i in range(2,ws_mgi.max_row+1)] 

    # Load dictionary file and open workbook
    workbook = openpyxl.load_workbook(dictionary_file_path)
    sheet = workbook.active

    ls_row_2 = [sheet.cell(row=i,column=2).value for i in range(2,sheet.max_row+1)]
    ls_row_9 = [sheet.cell(row=i,column=11).value for i in range(2,sheet.max_row+1)]

    
    length = len(ls_row_2) + 2

    # Modify the desired cell
    for i in range(length):
        K_cel = 'K' + str(i + 2)

        for k in range(len(ls_row_g_name)):
            try:
                if str(ls_row_g_name[k]).lower() == str(ls_row_2[i]).lower():
                    if ls_row_alias[k] is not None:
                        sheet[K_cel] = ls_row_alias[k]
                    else:
                        continue
            except:
                continue

    # Save the file
    workbook.save(file_path)


In [4]:
# LOAD START FILES

## PUBLICATION SOURCE
wb = openpyxl.load_workbook('../ifpan-GR-database-papers.xlsx')
ws = wb['adrenal-gland']

publication_geneName = [ws.cell(row=i,column=3).value for i in range(2,ws.max_row+1)]
Ensembl_id = [ws.cell(row=i,column=4).value for i in range(2,ws.max_row+1)]
direction = [ws.cell(row=i,column=6).value for i in range(2,ws.max_row+1)]
source_pimd = ws.cell(2, 2).value
organism_var = ws.cell(2, 7).value

## MGI
wb_1 = openpyxl.load_workbook(r'../MGI_EntrezGene.xlsx')
ws_1 = wb_1.active

ls_row_10 = [ws_1.cell(row=i,column=10).value for i in range(2,ws_1.max_row+1)]
ls_row_2 = [ws_1.cell(row=i,column=2).value for i in range(2,ws_1.max_row+1)]
ls_row_1 = [ws_1.cell(row=i,column=1).value for i in range(2,ws_1.max_row+1)]  

In [5]:
print(str(int(source_pimd)))
print(organism_var)

35904237
mouse


In [6]:
# LISTS

dictionary = {}
ls_geneDictionaries = []
ls_notResponse = []
ls_notResponse_after = []
ls_biomartParametersbyEnsembl = []


In [7]:
# DICTIONARY

# VARIABLES
gene_list_number = 25
gene_list_id = 'all_sig_genes_' + str(int(source_pimd))    # => cluster
source = 'PMID: '+ str(int(source_pimd))
organism = str(organism_var)
alias = ''



for i in range(len(publication_geneName)):
    gene_name = str(publication_geneName[i])
    direction_string = direction[i]   # => info
    
    if gene_name != '':
        index = i + 1

        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []

        ls_biomartParameters = biomartParameters(gene_name)

        if ls_biomartParameters:
            ensembl_gene_id_temp = [ls[0] for ls in ls_biomartParameters]
            ensembl_transcript_id_temp = [ls[1] for ls in ls_biomartParameters]
            refseq_mrna_temp = [ls[2] for ls in ls_biomartParameters]
        else:
            ensembl_gene_id = Ensembl_id[i]
            ls_biomartParametersbyEnsembl = biomartParametersbyEnsembl(ensembl_gene_id)

            ensembl_gene_id_temp = [ls[0] for ls in ls_biomartParametersbyEnsembl]
            ensembl_transcript_id_temp = [ls[1] for ls in ls_biomartParametersbyEnsembl]
            refseq_mrna_temp = [ls[2] for ls in ls_biomartParametersbyEnsembl]

        if not ensembl_gene_id_temp:
            ls_notResponse.append(gene_name)
            print(ls_notResponse)

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))


        # ORTHOLOGS
        ls_biomartHumanOrthologs = biomartHumanOrthologs(gene_name)
        
        if ls_biomartHumanOrthologs:
            hgnc_symbol = ls_biomartHumanOrthologs[0][0]
        else:
            ls_biomartHumanOrthologsbyEnsembl = biomartHumanOrthologsbyEnsembl(ensembl_gene_id)
            hgnc_symbol = ls_biomartHumanOrthologsbyEnsembl[0][0] if ls_biomartHumanOrthologsbyEnsembl else ''
            
        
        temp_gene_dictionary = gene_dictionary(index,
                                               gene_name,
                                               gene_list_number,
                                               gene_list_id,
                                               source,
                                               ensembl_gene_id,
                                               ensembl_transcript_id, 
                                               refseq_mrna,
                                               hgnc_symbol,
                                               organism,
                                               alias,
                                               direction_string)

        ls_geneDictionaries.append(temp_gene_dictionary)
        print(temp_gene_dictionary)





{'Index': 1, 'Gene_name': 'Per1', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000020893', 'Ensembl_transcript_id': 'ENSMUST00000166748|ENSMUST00000142392|ENSMUST00000021271|ENSMUST00000132462|ENSMUST00000102605|ENSMUST00000101004', 'RefSeq_mRNA_id': 'NM_011065|NM_001159367', 'Organism': 'mouse', 'HGNC_symbol': 'PER1', 'Alias': '', 'Info': 'up'}
{'Index': 2, 'Gene_name': 'Egr3', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000033730', 'Ensembl_transcript_id': 'ENSMUST00000223809|ENSMUST00000225200|ENSMUST00000223747|ENSMUST00000035908', 'RefSeq_mRNA_id': 'NM_001289927|NM_018781|NM_001289925', 'Organism': 'mouse', 'HGNC_symbol': 'EGR3', 'Alias': '', 'Info': 'up'}
{'Index': 3, 'Gene_name': 'Nr4a1', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000023

{'Index': 23, 'Gene_name': 'Ptgs2', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000032487', 'Ensembl_transcript_id': 'ENSMUST00000190784|ENSMUST00000035065', 'RefSeq_mRNA_id': 'NM_011198', 'Organism': 'mouse', 'HGNC_symbol': 'PTGS2', 'Alias': '', 'Info': 'up'}
{'Index': 24, 'Gene_name': 'Egr2', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000037868', 'Ensembl_transcript_id': 'ENSMUST00000130933|ENSMUST00000146986|ENSMUST00000145936|ENSMUST00000048289|ENSMUST00000105438|ENSMUST00000127820|ENSMUST00000145754', 'RefSeq_mRNA_id': 'NM_001373986|NM_001347458|NM_001373983|NM_010118|NM_001373985|NM_001373987', 'Organism': 'mouse', 'HGNC_symbol': 'EGR2', 'Alias': '', 'Info': 'up'}
{'Index': 25, 'Gene_name': 'Cfap43', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'EN

{'Index': 43, 'Gene_name': 'Noct', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000023087', 'Ensembl_transcript_id': 'ENSMUST00000023849|ENSMUST00000144826|ENSMUST00000167780|ENSMUST00000193018', 'RefSeq_mRNA_id': 'NM_009834', 'Organism': 'mouse', 'HGNC_symbol': 'NOCT', 'Alias': '', 'Info': 'up'}
{'Index': 44, 'Gene_name': 'Cnn2', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000004665', 'Ensembl_transcript_id': 'ENSMUST00000004784|ENSMUST00000105374', 'RefSeq_mRNA_id': 'NM_007725', 'Organism': 'mouse', 'HGNC_symbol': 'CNN2', 'Alias': '', 'Info': 'up'}
{'Index': 45, 'Gene_name': 'Dusp10', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000039384', 'Ensembl_transcript_id': 'ENSMUST00000048655|ENSMUST00000139839', 'RefSeq_mRNA_id': 'NM_022019', 'Organis

{'Index': 65, 'Gene_name': 'Plk3', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000028680', 'Ensembl_transcript_id': 'ENSMUST00000076859|ENSMUST00000126135|ENSMUST00000147730', 'RefSeq_mRNA_id': 'NM_001313916|NM_013807', 'Organism': 'mouse', 'HGNC_symbol': 'PLK3', 'Alias': '', 'Info': 'up'}
{'Index': 66, 'Gene_name': 'Rab20', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000031504', 'Ensembl_transcript_id': 'ENSMUST00000033900', 'RefSeq_mRNA_id': 'NM_011227', 'Organism': 'mouse', 'HGNC_symbol': 'RAB20', 'Alias': '', 'Info': 'up'}
{'Index': 67, 'Gene_name': 'Tgfb2', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000039239', 'Ensembl_transcript_id': 'ENSMUST00000194960|ENSMUST00000191766|ENSMUST00000045288|ENSMUST00000195201|ENSMUST00000194593|ENSMUST0

{'Index': 85, 'Gene_name': 'Coq10b', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000025981', 'Ensembl_transcript_id': 'ENSMUST00000185474|ENSMUST00000185918|ENSMUST00000087617|ENSMUST00000027125', 'RefSeq_mRNA_id': 'NM_001039710|NM_026424', 'Organism': 'mouse', 'HGNC_symbol': 'COQ10B', 'Alias': '', 'Info': 'up'}
{'Index': 86, 'Gene_name': 'Gm16091', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000087502', 'Ensembl_transcript_id': 'ENSMUST00000156664', 'RefSeq_mRNA_id': '', 'Organism': 'mouse', 'HGNC_symbol': '', 'Alias': '', 'Info': 'up'}
{'Index': 87, 'Gene_name': 'Rhob', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000054364', 'Ensembl_transcript_id': 'ENSMUST00000067384', 'RefSeq_mRNA_id': 'NM_007483', 'Organism': 'mouse', 'HGNC_symbol': 'RHOB

...

{'Index': 2247, 'Gene_name': 'Gnpda1', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000052102', 'Ensembl_transcript_id': 'ENSMUST00000235590|ENSMUST00000139885|ENSMUST00000153291|ENSMUST00000237313|ENSMUST00000129062|ENSMUST00000237487|ENSMUST00000063814', 'RefSeq_mRNA_id': 'NM_011937', 'Organism': 'mouse', 'HGNC_symbol': 'GNPDA1', 'Alias': '', 'Info': 'down'}
{'Index': 2248, 'Gene_name': 'Itpa', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000074797', 'Ensembl_transcript_id': 'ENSMUST00000103193|ENSMUST00000156382|ENSMUST00000126714|ENSMUST00000138963|ENSMUST00000133328', 'RefSeq_mRNA_id': 'NM_025922|NM_001362648', 'Organism': 'mouse', 'HGNC_symbol': 'ITPA', 'Alias': '', 'Info': 'down'}
{'Index': 2249, 'Gene_name': 'Nfatc4', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ense

{'Index': 2268, 'Gene_name': 'Cox6b1', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000036751', 'Ensembl_transcript_id': 'ENSMUST00000075738|ENSMUST00000208838|ENSMUST00000207851|ENSMUST00000208352', 'RefSeq_mRNA_id': 'NM_025628', 'Organism': 'mouse', 'HGNC_symbol': 'COX6B1', 'Alias': '', 'Info': 'down'}
{'Index': 2269, 'Gene_name': 'Rnf126', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000035890', 'Ensembl_transcript_id': 'ENSMUST00000218770|ENSMUST00000219189|ENSMUST00000047203', 'RefSeq_mRNA_id': 'NM_144528', 'Organism': 'mouse', 'HGNC_symbol': 'RNF126', 'Alias': '', 'Info': 'down'}
{'Index': 2270, 'Gene_name': 'Trim8', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000025034', 'Ensembl_transcript_id': 'ENSMUST00000026008|ENSMUST00000235934', 'Re

{'Index': 2287, 'Gene_name': 'Pdhb', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000021748', 'Ensembl_transcript_id': 'ENSMUST00000225735|ENSMUST00000022268|ENSMUST00000224690|ENSMUST00000225747|ENSMUST00000224466|ENSMUST00000225274|ENSMUST00000224864', 'RefSeq_mRNA_id': 'NM_024221', 'Organism': 'mouse', 'HGNC_symbol': 'PDHB', 'Alias': '', 'Info': 'down'}
{'Index': 2288, 'Gene_name': 'Atp5c1', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000025781', 'Ensembl_transcript_id': 'ENSMUST00000145530|ENSMUST00000130067|ENSMUST00000153554|ENSMUST00000026887|ENSMUST00000114896|ENSMUST00000114897|ENSMUST00000139810', 'RefSeq_mRNA_id': 'NM_001112738|NM_020615|NM_001374641', 'Organism': 'mouse', 'HGNC_symbol': 'ATP5F1C', 'Alias': '', 'Info': 'down'}
{'Index': 2289, 'Gene_name': 'Cst3', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_

{'Index': 2306, 'Gene_name': 'Cox7a2l', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000024248', 'Ensembl_transcript_id': 'ENSMUST00000167741|ENSMUST00000025095|ENSMUST00000235085|ENSMUST00000234258', 'RefSeq_mRNA_id': 'NM_009187|NM_001159529', 'Organism': 'mouse', 'HGNC_symbol': 'COX7A2L', 'Alias': '', 'Info': 'down'}
{'Index': 2307, 'Gene_name': 'Tpt1', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000060126', 'Ensembl_transcript_id': 'ENSMUST00000134040|ENSMUST00000150047|ENSMUST00000154533|ENSMUST00000128889|ENSMUST00000151751|ENSMUST00000125146|ENSMUST00000110894|ENSMUST00000142061', 'RefSeq_mRNA_id': 'NM_009429', 'Organism': 'mouse', 'HGNC_symbol': '', 'Alias': '', 'Info': 'down'}
{'Index': 2308, 'Gene_name': 'Coq10a', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensemb

{'Index': 2326, 'Gene_name': 'Pim1', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000024014', 'Ensembl_transcript_id': 'ENSMUST00000234665|ENSMUST00000024811|ENSMUST00000234650|ENSMUST00000234263|ENSMUST00000234294|ENSMUST00000234478|ENSMUST00000234270', 'RefSeq_mRNA_id': 'NM_008842|NM_001364913', 'Organism': 'mouse', 'HGNC_symbol': 'PIM1', 'Alias': '', 'Info': 'down'}
{'Index': 2327, 'Gene_name': 'Ndufb6', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000071014', 'Ensembl_transcript_id': 'ENSMUST00000095128|ENSMUST00000108108', 'RefSeq_mRNA_id': 'NM_001033305', 'Organism': 'mouse', 'HGNC_symbol': 'NDUFB6', 'Alias': '', 'Info': 'down'}
{'Index': 2328, 'Gene_name': 'Bloc1s4', 'Gene_list_number': 25, 'Gene_list_index': 'all_sig_genes_35904237', 'Source': 'PMID: 35904237', 'Ensembl_gene_id': 'ENSMUSG00000060708', 'Ensembl_transcri

In [8]:
print(ls_notResponse)

['Gm26809', 'Gm22918', 'Gm23689', 'NA', 'Ptprg', 'NA', 'NA', 'Dhrsx', 'NA']


In [9]:
# SCORES 

### from DICTIONARY to file
data = ls_geneDictionaries
df = pd.DataFrame.from_dict(data)
df.to_excel(r'.\Dictionary.xlsx', index=False)

### from ls_notResponse to file
data1 = ls_notResponse
df1 = pd.DataFrame.from_dict(data1)
df1.to_excel(r'.\notResponse.xlsx', index=False)

# LOAD 
# notRESPONSE
wb_2 = openpyxl.load_workbook(r'.\notResponse.xlsx')
ws_2 = wb_2.active
ls_notResponse_v2 = [ws_2.cell(row=i,column=1).value for i in range(2,ws_2.max_row+1)]

# MGI
wb_1 = openpyxl.load_workbook(r'..\MGI_EntrezGene.xlsx')
ws_1 = wb_1.active

ls_row_10 = [ws_1.cell(row=i,column=10).value for i in range(2,ws_1.max_row+1)]
ls_row_2 = [ws_1.cell(row=i,column=2).value for i in range(2,ws_1.max_row+1)]


### from alias_and_official to file
data2 = alias_and_official(ls_notResponse, ls_row_10, ls_row_2)
df2 = pd.DataFrame.from_dict(data2)
df2.to_excel(r'.\responsewithMGI.xlsx', index=False)


# LOAD responsewithMGI
wb_3 = openpyxl.load_workbook(r'.\responsewithMGI.xlsx')
ws_3 = wb_3.active
mgi_id = [ws_3.cell(row=i,column=3).value for i in range(3,ws_3.max_row+1)]
gene_name_v2 = [ws_3.cell(row=i,column=1).value for i in range(3,ws_3.max_row+1)]

In [10]:
print (ls_notResponse)

['Gm26809', 'Gm22918', 'Gm23689', 'NA', 'Ptprg', 'NA', 'NA', 'Dhrsx', 'NA']


In [11]:
# SECOND DICTIONARY

for i in range(len(mgi_id)):
    mgi_ID = mgi_id[i]
    gene_name = gene_name_v2[i]
    print(mgi_ID)
    
    if gene_name != '':

        # variables
        alias = ''

        ensembl_gene_id_temp = []
        ensembl_transcript_id_temp = []
        refseq_mrna_temp = []
        

        ls_biomartParameters_mgi = biomartParameters_mgi(mgi_ID)

        if ls_biomartParameters_mgi:
            ensembl_gene_id_temp = [ls[0] for ls in ls_biomartParameters_mgi]
            ensembl_transcript_id_temp = [ls[1] for ls in ls_biomartParameters_mgi]
            refseq_mrna_temp = [ls[2] for ls in ls_biomartParameters_mgi]
        else:
            ls_notResponse_after.append(mgi_ID)

        ensembl_gene_id = '|'.join(list(set(filter(None, ensembl_gene_id_temp))))
        ensembl_transcript_id = '|'.join(list(set(filter(None, ensembl_transcript_id_temp))))
        refseq_mrna = '|'.join(list(set(filter(None, refseq_mrna_temp))))

        # ORTHOLOGS
        ls_biomartHumanOrthologs_mgi = biomartHumanOrthologs_mgi(mgi_ID)

        if ls_biomartHumanOrthologs_mgi:
            hgnc_symbol = ls_biomartHumanOrthologs_mgi[0][0]
        else:
            hgnc_symbol = ''

        for k in range(len(ls_geneDictionaries)):
            temp = str(ls_geneDictionaries[k]['Gene_name']).lower()

            if temp == gene_name:
                print('response')
                ls_geneDictionaries[k]['Ensembl_gene_id'] = ensembl_gene_id
                ls_geneDictionaries[k]['Ensembl_transcript_id'] = ensembl_transcript_id
                ls_geneDictionaries[k]['RefSeq_mRNA_id'] = refseq_mrna
                ls_geneDictionaries[k]['HGNC_symbol'] = hgnc_symbol
                


In [12]:
# SCORES 

### from SECOND DICTIONARY to file

data3 = ls_geneDictionaries
df3 = pd.DataFrame.from_dict(data3)
df3.to_excel(r'.\secondDictionary.xlsx', index=False)
df3.to_csv(r'.\secondDictionary.tsv', sep="\t")

data4 = ls_notResponse_after
df4 = pd.DataFrame.from_dict(data4)
df4.to_excel(r'.\ls_notResponse_after.xlsx', index=False)



In [13]:
# Add ALIAS

mgi_file_path = r'..\MGI_EntrezGene.xlsx'
dictionary_file_path = r'.\secondDictionary.xlsx'
save_file_path = r'.\withAlias.xlsx'


updateCellswithAlias(mgi_file_path, dictionary_file_path, save_file_path)


In [14]:
# Add INFO

def updateCellswithINFO(source_file_path, source_sheet, alias_file_path, info_file_path):
    
    wb_info = openpyxl.load_workbook(source_file_path)
    ws_info = wb_info[source_sheet]
    ls_row_g_name = [ws_info.cell(row=i, column=3).value for i in range(2, ws_info.max_row + 1)]


    wb_alias = openpyxl.load_workbook(alias_file_path)
    ws_alias = wb_alias.active
    ls_row_2 = [ws_alias.cell(row=i, column=2).value for i in range(2, ws_alias.max_row + 1)]


    matched_indices = set()               # to keep track of matched id

    for j, (g_name, row_2) in enumerate(zip(ls_row_g_name, ls_row_2), start=2):
        if j not in matched_indices and str(g_name).lower() == str(row_2).lower():
            print(str(g_name).lower(), str(row_2).lower())
            L_cel = 'L' + str(j)
            tempInfo = str(ws_alias[L_cel].value)
            matched_indices.add(j)        # add matched index to set - avoid duplicate

### the number and type of items in the INFO column depends on the source publication


            ws_alias[L_cel] = ''
            ws_alias[L_cel] = ('Regulation:' + str(tempInfo).upper() + '|'
                            + 'Log2Ratio:' + str(ws_info['E' + str(j)].value) + '|'
                            + 'tissue:' + str(ws_info['H' + str(j)].value) + '|'
                            + 'cell:' + str(ws_info['I' + str(j)].value) + '|'
                            + 'enviroroment:' + str(ws_info['J' + str(j)].value) + '|'
                            + 'treatment:' + str(ws_info['K' + str(j)].value) + '|'
                            + 'dose:' + str(ws_info['L' + str(j)].value) + '|'
                            + 'time:' + str(ws_info['M' + str(j)].value) + '|'
                            + 'FDR_threshold:' + str(ws_info['N' + str(j)].value) + '|'
                            + 'method:' + str(ws_info['O' + str(j)].value) + '|'
                            + 'FDR:' + str(ws_info['P' + str(j)].value) + '|'
                            + 'statistics:' + str(ws_info['Q' + str(j)].value))

        
        


    # Save the file
    wb_alias.save(info_file_path)



source_file_path = r'..\ifpan-GR-database-papers.xlsx'
source_sheet = 'adrenal-gland'
alias_file_path = '.\withAlias.xlsx'
info_file_path = r'.\withINFO.xlsx'

updateCellswithINFO(source_file_path, source_sheet, alias_file_path, info_file_path)


per1 per1
egr3 egr3
nr4a1 nr4a1
nedd9 nedd9
srf srf
dusp1 dusp1
gadd45g gadd45g
btg2 btg2
arc arc
nfil3 nfil3
lif lif
fosb fosb
il17ra il17ra
csrnp1 csrnp1
maff maff
dusp8 dusp8
amotl2 amotl2
filip1l filip1l
atf3 atf3
trib1 trib1
cited2 cited2
tent5b tent5b
ptgs2 ptgs2
egr2 egr2
cfap43 cfap43
rusc2 rusc2
irak2 irak2
zyx zyx
klf7 klf7
jun jun
9330159f19rik 9330159f19rik
cyr61 cyr61
fhl2 fhl2
klf6 klf6
gm44164 gm44164
cttnbp2nl cttnbp2nl
mafk mafk
lmod1 lmod1
ifrd1 ifrd1
arid5a arid5a
bach1 bach1
lmcd1 lmcd1
noct noct
cnn2 cnn2
dusp10 dusp10
serpine1 serpine1
neat1 neat1
skil skil
pdp1 pdp1
zfp281 zfp281
six4 six4
gadd45b gadd45b
traf5 traf5
plaur plaur
zc3h12c zc3h12c
tuft1 tuft1
elmsan1 elmsan1
errfi1 errfi1
mthfd2 mthfd2
mob3a mob3a
aff1 aff1
nfkbia nfkbia
bhlhe41 bhlhe41
zfp36 zfp36
plk3 plk3
rab20 rab20
tgfb2 tgfb2
actg1 actg1
nr4a3 nr4a3
usp53 usp53
npas4 npas4
gm17501 gm17501
gm43909 gm43909
prkab2 prkab2
mir22hg mir22hg
zfp568 zfp568
akap12 akap12
fgd3 fgd3
vmp1 vmp1
srsf5 srsf5


In [ ]:
                                ''' + '|'
                                 + '|'
                                +'TTEST.NEURO.ALD:' + str(ws_mgi[S_cel].value) + '|'
                                +'TTEST.NEURO.DHT:' + str(ws_mgi[T_cel].value) + '|'
                                +'TTEST.NEURO.PRG:' + str(ws_mgi[U_cel].value) + '|'
                                +'TTEST.NEURO.ESR:' + str(ws_mgi[V_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.DEX:' + str(ws_mgi[W_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.ALD:' + str(ws_mgi[X_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.DHT:' + str(ws_mgi[Y_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.PRG:' + str(ws_mgi[Z_cel].value) + '|'
                                +'FDR_TTEST.ASTRO.ESR:' + str(ws_mgi[AA_cel].value) + '|'
                                +'FDR_TTEST.NEURO.DEX:' + str(ws_mgi[AB_cel].value) + '|'
                                +'FDR_TTEST.NEURO.ALD:' + str(ws_mgi[AC_cel].value) + '|'
                                +'FDR_TTEST.NEURO.DHT:' + str(ws_mgi[AD_cel].value) + '|'
                                +'FDR_TTEST.NEURO.PRG:' + str(ws_mgi[AE_cel].value) + '|'
                                +'FDR_TTEST.NEURO.ESR:' + str(ws_mgi[AF_cel].value) + '|'
                                +'FOLD.ASTRO.DEX:' + str(ws_mgi[AG_cel].value) + '|'
                                +'FOLD.ASTRO.ALD:' + str(ws_mgi[AH_cel].value) + '|'
                                +'FOLD.ASTRO.DHT:' + str(ws_mgi[AI_cel].value) + '|'
                                +'FOLD.ASTRO.PRG:' + str(ws_mgi[AJ_cel].value) + '|'
                                +'FOLD.ASTRO.ESR:' + str(ws_mgi[AK_cel].value) + '|'
                                +'FOLD.NEURO.DEX:' + str(ws_mgi[AL_cel].value) + '|'
                                +'FOLD.NEURO.ALD:' + str(ws_mgi[AM_cel].value) + '|'
                                +'FOLD.NEURO.DHT:' + str(ws_mgi[AN_cel].value) + '|'
                                +'FOLD.NEURO.PRG:' + str(ws_mgi[AO_cel].value) + '|'
                                +'FOLD.NEURO.ESR:' + str(ws_mgi[AP_cel].value))'''